In [1]:
import yfinance
import polars as pl
import duckdb
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px


In [2]:
df = pl.read_parquet('..\\data\\candle_data.parquet')

In [3]:
df = pl.read_parquet('..\\data\\candle_data.parquet')
close_std = df['close'].std()
short_window = 6
long_window = 12

# this was fixed in the api aggregation
df = df.with_columns(returns=pl.col('close').pct_change(),
                     log_returns=(pl.col('close').pct_change().log1p()) * 100,
                     ema_long = pl.col('close').ewm_mean(span=long_window, min_periods=long_window, ignore_nulls=False, adjust=False), #60 minute exp moving avg
                     ema_short = pl.col('close').ewm_mean(span=short_window, min_periods=short_window, ignore_nulls=False, adjust=False), #30 minute exp moving avg
                     rolling_avg = pl.col('close').rolling_mean(window_size=20),
                     lower_bb=pl.col('close') - close_std,
                     upper_bb=pl.col('close') + close_std

                ) 

In [4]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Line(x=df['tradetime'], y=df['close'], name="Close"),
    secondary_y=False,
)

fig.add_trace(
    go.Line(x=df['tradetime'], y=df['ema_long'], name=f'ema({long_window*5}m)')
)

fig.add_trace(
    go.Line(x=df['tradetime'], y=df['ema_short'], name=f'ema({short_window*5}m)')
)


# Add figure title
fig.update_layout(
    title_text="BTC-USD w/ Indicators"
)

# Set x-axis title
fig.update_xaxes(title_text="Tradetime (5 min. intervals)")

# Set y-axes titles
fig.update_yaxes(title_text="BTC-USD ($)", secondary_y=False)
# fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)

fig.show()

c:\Users\grant\Documents\Projects\crypto-datawarehouse\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [5]:
signals_df = df.with_columns(signal = pl.when((pl.col("ema_short") > pl.col('ema_long')) & (pl.col('ema_short').shift(-1) <= pl.col('ema_long').shift(-1))) # bullish
        .then(pl.lit('buy'))
        .when((pl.col("ema_short") < pl.col('ema_long')) & (pl.col('ema_short').shift(-1) >=  pl.col('ema_long').shift(-1)))
        .then(pl.lit('sell'))
        .otherwise(pl.lit('hold'))).select(pl.col('tradetime'), pl.col('close'), pl.col('ema_long'), pl.col('ema_short'), pl.col('signal'))

In [6]:
signals_df.filter(pl.col('signal') != 'hold')

filtered_signals = signals_df.filter(pl.col('signal') != 'hold')

In [20]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Line(x=df['tradetime'], y=df['close'], name="Close"),
    secondary_y=False,
)

fig.add_trace(
    go.Line(x=df['tradetime'], y=df['ema_long'], name=f'ema({long_window*5}m)')
)

fig.add_trace(
    go.Line(x=df['tradetime'], y=df['ema_short'], name=f'ema({short_window*5}m)')
)

def color_map(val):
    if val == 'buy':
        return 'green'
    elif val == 'sell':
        return 'red'
    else:
        return 'gray'

fig.add_trace(
    go.Scatter(x=filtered_signals['tradetime'],
                y = filtered_signals['close'],
                mode="markers", 
                customdata=filtered_signals['signal'],
                marker=dict(color=[color_map(val) for val in filtered_signals['signal']]),
                name = 'BUY/SELL Signal'
                )
            )


# Add figure title
fig.update_layout(
    title_text="BTC-USD w/ Indicators & Markers"
)

# Set x-axis title
fig.update_xaxes(title_text="Tradetime (5 min. intervals)")

# Set y-axes titles
fig.update_yaxes(title_text="BTC-USD ($)", secondary_y=False)
# fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)

fig.show()

c:\Users\grant\Documents\Projects\crypto-datawarehouse\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


